# Notes on Phase Diversity

Reference: https://www.sciencedirect.com/science/article/pii/S0143816620304553

## Image model

The image amplitude at defocus $0$, $i_0$, is the convolution of the object amplitude $o$ and the broadband PSF at defocus $0$, $h_0$ (noise is neglected):

$$i_0(x, y, \lambda) = o(x, y, \lambda) * h_0(x, y, \lambda)$$

In the case of Euclid, $o$ is a star modeled as a point source (unknown position $(x_s, y_s)$) with spectrum $S$:

$$
\begin{align}
o(x_s, y_s, \lambda) &= S(\lambda) \\
o(x, y, \lambda) &= 0 \text{ for } (x, y) \neq (x_s, y_s)
\end{align}
$$

## PSF model

The broadband PSF is the integral of the monochromatic PSFs over $\lambda$:

$$h_0 = \int_\lambda h_{\lambda,0} \, S(\lambda) d\lambda$$

Monochromatic PSFs are computed as the Fourier transform of the complex pupil function:

$$h_{\lambda,0}(x, y) = |\mathcal{F}^{-1}(P(u, v) \, \exp(-j2\pi/\lambda\cdot\Phi_0(u, v)))|^2$$

where:
* $P$ is the $1024 \times 1024$ pupil mask, values of which are reals in $(0, 1)$ for antialiasing at edges, but are mostly 0's and 1's;
* $\Phi_0$ is the WFE at defocus 0.

## WFE model

The WFE is decomposed into Zernike polynomials:

$$\Phi_0 = \sum_i \alpha_i Z_i$$

where the Zernike polynomial $Z_i$ is precomputed as a $1024 \times 1024$ matrix, and the Zernike coefficients $\alpha_i$ are the unknowns.

## Phase diversity

Same equations hold at defocus $\delta$:

$$
\begin{align}
h_{\lambda,\delta}(x, y) &= |\mathcal{F}^{-1}(P(u, v) \, \exp(-j2\pi/\lambda\cdot\Phi_\delta(u, v)))|^2 \\
\Phi_\delta &= \sum_i \beta_i Z_i
\end{align}
$$

The defocus mechanism is known well-enough to assume that the $\beta_i$'s equal the $\alpha_i$'s except for $i = 4$ and $i = 11$.

## Error function

An error function $E$ is introduced in Fourier space, to be minimized:

$$
E = \sum_{(u,v)}\frac{\|I_0(u,v) - (a_0 H_0(u,v) + b_0)\|^2}{\sigma_0^2(u,v)}
+ \frac{\|I_\delta(u,v) - (a_\delta H_\delta(u,v) + b_\delta)\|^2}{\sigma_\delta^2(u,v)}
$$

where uppercase letters denote the Fourier transforms of their lowercase counterparts.

The error function can be extended to more defocus values.

## Unknowns

From the above equations, the unknowns are:

- The position of the stars: $(x_s, y_s)$ for $s = 1..1000$;
- The Zernike coefficients: $\alpha_i$ for $i = 1..40$ as well as $\beta_4$ and $\beta_{11}$;
- The PSF normalization coefficients: $a_0, b_0, a_\delta, b_\delta$.

## Computation

Given:
* $L$ the number of wavelengths ($L \in [?]$);
* $M$ the number of Zernike polynomials ($L \in [20, 40]$);
* $N$ the number of input images (star thumbnails, $N \in [100, 1000]$);

the error is computed $L \times (M + 1) \times N$ times in order to build the Jacobian matrix (one-step derivatives).

In terms of Fourier transform, this means:
* One direct transform to estimate $h$ from $P$ and the $a_i$'s;
* One direct transform to estimate $H$ from $h$.